# Claude 3 as Economic Analyst

This notebook creates a step-by-step notebook on how to recreate the [Claude 3 Economic Analyst from the Youtube Demo](https://www.youtube.com/watch?v=sjL6Gl6ZIqs)

## Setup
### Install dependencies (or alternatively, install on env)

In [ ]:
# Install the necessary libraries
%pip install anthropic selenium plotly

### Import Anthropic and initialize client
You can get your API key from: [Anthropic Console - API Key](https://console.anthropic.com/settings/keys)

In [ ]:
# Import required libraries
from anthropic import Anthropic
import re

# Set up the Anthropic API client
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

### Construct System Prompt with Function Definitions and Parameters

Tool formatting

In [ ]:
def construct_format_tool_for_claude_prompt(name, description, parameters):
    constructed_prompt = (
        "<tool_description>\n"
        f"<tool_name>{name}</tool_name>\n"
        "<description>\n"
        f"{description}\n"
        "</description>\n"
        "<parameters>\n"
        f"{construct_format_parameters_prompt(parameters)}\n"
        "</parameters>\n"
        "</tool_description>"
    )
    return constructed_prompt

def construct_format_parameters_prompt(parameters):
    constructed_prompt = "\n".join(f"<parameter>\n<name>{parameter['name']}</name>\n<type>{parameter['type']}</type>\n<description>{parameter['description']}</description>\n</parameter>" for parameter in parameters)

    return constructed_prompt


Tool names, definitions, and paramaters definitions

In [ ]:
# Web View Tool Definition
tool_name_1 = "web_view"
tool_description_1 = """Can look up information on the web by going to a URL and takes a screenshot of the page.
    Returns the screenshot as a PNG image.
    Raises ValueError if the provided location cannot be found."""
parameters_1 = [
    {
        "name": "url",
        "type": "string",
        "description": "The URL of the page to screenshot."
    }
]

# Python Interpreter Tool Definition
tool_name_2 = "python_interpreter_tool"
tool_description_2 = """Receives python code and returns the output of the code. It can run code for creating visualizations, data analysis, and similar other tasks. This tool is useful for running python code with libraries which are available in the default python interpreter.
    Raises ValueError if the provided location cannot be found."""
parameters_2 = [
    {
        "name": "code",
        "type": "string",
        "description": "Entire Python code to be executed in a Python interpreter."
    }
]

Create tool formattig with defined tools

In [ ]:
tool_1 = construct_format_tool_for_claude_prompt(tool_name_1, tool_description_1, parameters_1)
tool_2 = construct_format_tool_for_claude_prompt(tool_name_2, tool_description_2, parameters_2)
print(tool_1, tool_2)

Create system prompt with tools for system message

In [ ]:
def construct_tool_use_system_prompt(tools):
    tool_use_system_prompt = (
        "In this environment you have access to a set of tools you can use to answer the user's question.\n"
        "\n"
        "You may call them like this:\n"
        "<function_calls>\n"
        "<invoke>\n"
        "<tool_name>$TOOL_NAME</tool_name>\n"
        "<parameters>\n"
        "<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>\n"
        "...\n"
        "</parameters>\n"
        "</invoke>\n"
        "</function_calls>\n"
        "\n"
        "Here are the tools available:\n"
        "<tools>\n"
        + '\n'.join([tool for tool in tools]) +
        "\n</tools>"
    )
    return tool_use_system_prompt

system_prompt = construct_tool_use_system_prompt([tool_1, tool_2])
print(system_prompt)

## Define Tool Functions

### Define Web View Tool

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import base64

def web_view(url):
    # Configure Chrome options for headless browsing
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Set the desired window size
    window_size = (256, 256)
    chrome_options.add_argument(f"--window-size={window_size[0]},{window_size[1]}")

    # Create a new Chrome WebDriver instance with the configured options
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Navigate to the URL
        driver.get(url)

        # Capture the screenshot of the webpage
        screenshot = driver.get_screenshot_as_base64()

        return screenshot

    finally:
        # Quit the WebDriver
        driver.quit()

### Define Python Interpreter Tool

In [ ]:
def python_interpreter(code: str):
    """
    Executes the provided Python code string.

    Args:
        code (str): The Python code to execute.
    """
    try:
        # Dynamically execute the provided code
        exec(code, {})
    except Exception as e:
        # Exception handling can be adjusted based on requirements
        print(f"An error occurred during execution: {e}")

## Query Claude 3

### Create 1st Query - Fetch Web Page Screenshot

In [ ]:
user_message = {
    "role": "user", 
    "content": "Look up the GDP trends for the US and write a markdown table of the estimates by year since 2015 to the nearest 0.5 trillion dollars."
}

Call Claude 3

In [ ]:
function_calling_message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    messages=[user_message],
    system=system_prompt,
    stop_sequences=["\n\nHuman:", "\n\nAssistant", "</function_calls>"]
).content[0].text
print(function_calling_message)

Extract URL returned by Claude 3 and run Web View tool with URL

In [ ]:
def extract_between_tags(tag: str, string: str):
    url = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    return url[0]

url = extract_between_tags("url", function_calling_message)

print(url)

result = web_view(url)
print(result)

Construct successful function run prompt including function result

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>" 
            for res in invoke_results
        ) + "\n</function_results>"
    )
    
    return constructed_prompt

formatted_results = [{
    'tool_name': 'web_view',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

Pass Screenshot obtained from the Web View Tool using provided URL to Claude 3

In [ ]:
partial_assistant_message = function_calling_message + "</function_calls>" + function_results

final_message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    messages=[
        user_message,
        {
            "role": "assistant",
            "content": partial_assistant_message
        }
    ],
    system=system_prompt
).content[0].text
print(final_message)

### Create 2nd Query for Claude 3 - Interactive Chart

In [ ]:
user_message_2 = {
    "role": "user", 
    "content": "Make an interactive plot of this data, with a tooltip every year describing interesting economic events."
}

Create second prompt passing in the results from the previous query

In [ ]:
function_calling_message_2 = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    messages=[
        user_message_2,
        {
            "role": "assistant",
            "content": final_message
        }],
    system=system_prompt,
    stop_sequences=["\n\nHuman:", "\n\nAssistant", "</function_calls>"]
).content[0].text
print(function_calling_message_2)

Extract Python code returned by Claude 3 and run Python Interpreter Tool with code

In [ ]:
def extract_between_tags(tag: str, string: str):
    code = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    return code[0]

code = extract_between_tags("code", function_calling_message_2)

print(code)

result = python_interpreter(code)
print(result)